<a href="https://colab.research.google.com/github/AI-Junction/Lung-Cancer-Detection/blob/master/LungCancerDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 245kB 4.8MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

#filename = "/content/.kaggle/kaggle.json"
#filename = "/.kaggle/kaggle.json"
filename = "kaggle.json"
#os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!ls -lha kaggle.json

---x-wx--T 1 root root 65 Dec  4 17:50 kaggle.json


In [4]:
os.path.isdir("/root/.kaggle")

False

In [5]:
dir_kaggle = "/root/.kaggle"
if not os.path.isdir(dir_kaggle):
  !mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle/
os.path.isdir(dir_kaggle)

True

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
! pip install dicom

    100% |████████████████████████████████| 768kB 5.2MB/s 


In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection



dsb_data_dir = "drive/My Drive/All ML Datasets/AllLuna16Data"

/usr/local/lib/python3.6/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [0]:
def plot_3d(image, threshold=-300):

    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    #p = first_patient_pixels.transpose(2,1,0)
    
    p = pix_resampled.transpose(2,1,0)
    verts, faces, norm, val  = measure.marching_cubes(p, threshold)
    
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    
    plt.show()


In [0]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None


In [18]:
#INPUT_FOLDER = os.getcwd()
#print(INPUT_FOLDER)
path = os.path.join(dsb_data_dir, "R_004/06-30-1997-Diagnostic Pre-Surgery Contrast Enhanced CT-71813/3- NONE -29295")
patients = os.listdir(path)
print(patients)
patients.sort()


['000000.dcm', '000001.dcm', '000002.dcm', '000005.dcm', '000003.dcm', '000004.dcm', '000006.dcm', '000007.dcm', '000008.dcm', '000009.dcm', '000011.dcm', '000013.dcm', '000012.dcm', '000010.dcm', '000016.dcm', '000017.dcm', '000014.dcm', '000019.dcm', '000018.dcm', '000015.dcm', '000021.dcm', '000022.dcm', '000024.dcm', '000023.dcm', '000029.dcm', '000025.dcm', '000020.dcm', '000026.dcm', '000028.dcm', '000027.dcm', '000030.dcm', '000036.dcm', '000034.dcm', '000031.dcm', '000037.dcm', '000033.dcm', '000035.dcm', '000038.dcm', '000032.dcm', '000039.dcm', '000040.dcm', '000045.dcm', '000041.dcm', '000046.dcm', '000044.dcm', '000043.dcm', '000042.dcm', '000048.dcm', '000047.dcm', '000049.dcm', '000050.dcm', '000051.dcm', '000053.dcm', '000052.dcm', '000057.dcm', '000058.dcm', '000054.dcm', '000059.dcm', '000055.dcm', '000056.dcm', '000062.dcm', '000065.dcm', '000066.dcm', '000060.dcm', '000067.dcm', '000061.dcm', '000064.dcm', '000063.dcm']
